# Notebook para asignar los embeddings de las respuestas de acuerdo con la implementación de GloVe preentrenado

**Cargar librerias**

In [ ]:
import gensim
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import sent_tokenize


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\angel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\angel\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


### Cargar el embedding preentrenado (GloVe en espanol)

In [2]:
def load_glove_model(glove_file):
    print("Loading GloVe Model...")
    glove_model = {}
    with open(glove_file, 'r', encoding="utf-8") as f:
        for line in f:
            split_line = line.split()
            word = split_line[0]
            embedding = np.array([float(val) for val in split_line[1:]])
            glove_model[word] = embedding
    print("Done.", len(glove_model), "words loaded!")
    return glove_model

glove_file = 'glove_6B/glove-sbwc.i25.vec'
glove_model = load_glove_model(glove_file)


Loading GloVe Model...
Done. 855380 words loaded!


### Importar dataset 

(Solamente hacer si se desea probar otro arreglo aleatorio diferente)

In [ ]:
#data= pd.read_csv('datos/train_poll_v1.csv')

Mezclar los valores

In [ ]:
# Scramble the values of the data frame and reset the index
#data = data.sample(frac=1).reset_index(drop=True)

In [ ]:
# save data
#data.to_csv('datos/train_poll_v1s1.csv', index=False)

### Generar los vectores GloVe

Tokenizar las palabras de cada una de las oraciones de cada respuesta

In [5]:
data= pd.read_csv('datos/train_poll_v1s1.csv')
respuestas = list(data['respuestas'].values)
#counter=0 Este counter se utiliza si se desea limitar el número de oraciones en cada respuesta
indice_extras=[]
respuestas_token = []
for i in respuestas:
    #Tokenize the text into sentences
    sentences = sent_tokenize(i)
    #if len(sentences)<3 or len(sentences)>100:
     #   indice_extras.append(counter)
    #else:
        # Tokenize each sentence into words
    tokenized_sentences = [word_tokenize(sentence) for sentence in sentences]
    respuestas_token.append(tokenized_sentences)
    

Ejemplo de tokenizacion

In [16]:
respuestas_token[7]

[['Tener',
  'la',
  'disciplina',
  'y',
  'constancia',
  'que',
  'se',
  'necesita',
  'para',
  'tener',
  'un',
  'buen',
  'desempeño',
  'en',
  'el',
  'salón']]

In [17]:
# Load stopwords
stop_words = set(stopwords.words('spanish'))
#Esta instrucción se puede comentar si no se desea eliminar una palabra repetitiva
#stop_words.add("(Reuters)")

### Generar un embedding para cada oracion de la respuesta

(Haciendo un promedio de los embeddings de cada palabra en una oración)

In [18]:
def sentence_embedding(sentence_tokens, embeddings, stop_words):
    embedding_dim = 300  # GloVe 50d
    sentence_vector = np.zeros(embedding_dim)
    word_count = 0
    
    for word in sentence_tokens:
        word = word.lower()
        if word not in stop_words and word in embeddings:
            sentence_vector += embeddings[word]
            word_count += 1
            
    if word_count > 0:
        sentence_vector /= word_count  # Optionally, normalize by number of words
    
    return sentence_vector

In [19]:
# Calculate embeddings for each sentence, ignoring stopwords
gloVe_Embedding=[]
for tokenized_sentences in respuestas_token:

    sentence_embeddings = [sentence_embedding(sentence, glove_model, stop_words) for sentence in tokenized_sentences]
    sentence_embeddings = np.array(sentence_embeddings)
    gloVe_Embedding.append(sentence_embeddings)

In [ ]:
#save the values
np.save('datos/encuestav2_gloVe_Values.npy', gloVe_Embedding)